In [ ]:
!pip install timeout-decorator
!pip install scipy==1.8.0
!pip install networkx==3.0
!pip install cvxpy==1.2.3

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spi

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.linalg import cholesky
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.algorithms.community import greedy_modularity_communities
import cvxpy
import pandas as pd
import os
import timeout_decorator
from networkx.algorithms.community import kernighan_lin_bisection
os.chdir("/content/drive/MyDrive/Colab Notebooks/研究")

In [ ]:
@timeout_decorator.timeout(1)
def create_graph(seed_i):
  #LFR Benchmarkグラフ
  G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.9, average_degree=4, min_community=50, seed=seed_i)
  #G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.1, average_degree=4, min_community=50, seed=10) 
  #G = nx.LFR_benchmark_graph(100, 2, 1.1, 0.1, average_degree=15, max_degree=50, min_community=10, max_community=100)
  #G = nx.LFR_benchmark_graph(100, 5, 1.5, 0.1, average_degree=10, min_community=20, max_community=50)
  #G = nx.LFR_benchmark_graph(100, 3, 1.5, 0.1, average_degree=5, min_community=20)
  return G

In [ ]:
#seed_raw = [i for i in range(10000)]
#seed_correct = []
#for i in seed_raw:
#  if len(seed_correct) >= 100:
#    break
#  try:
#    G = create_graph(i)
#    seed_correct.append(i)
#  except nx.ExceededMaxIterations:
#    continue
#  except timeout_decorator.TimeoutError:
#    continue
  
#print(seed_correct)

In [ ]:
#len(seed_correct)

In [ ]:
mean = 0
colors = ['red', 'blue', 'green', 'yellow', 'pink', 'crimson', 'orange']
list2 =[]

seed_correct = [6, 7, 10, 29, 39, 75, 81, 104, 136, 166, 171, 235, 248, 253, 266, 332, 337, 367, 394, 399, 400, 409, 429, 430, 449, 477, 479, 486, 487, 494, 500, 501, 521, 535, 540, 541, 552, 579, 582, 594, 600, 602, 629, 653, 657, 673, 680, 688, 689, 703, 709, 730, 738, 762, 773, 775, 852, 867, 875, 878, 880, 889, 904, 908, 918, 933, 945, 950, 955, 970, 971, 996, 1048, 1051, 1054, 1067, 1089, 1091, 1109, 1120, 1137, 1167, 1189, 1207, 1245, 1334, 1349, 1353, 1358, 1368, 1371, 1376, 1410, 1413, 1420, 1467, 1469, 1476, 1479, 1480]
for seeds in seed_correct:
  G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.1, average_degree=4, min_community=50, seed=seeds)
  pos = nx.spring_layout(G)

  N = G.number_of_nodes()
  M = G.number_of_edges()
  V = np.zeros((N, N))
  Vp = np.zeros((N, N))
  Vn = np.zeros((N, N))
  Lam = np.zeros((N, N))
  Lamp = np.zeros((N, N))
  Lamn = np.zeros((N, N))
  Np = 0
  Nn = 0
  A = np.array(nx.adjacency_matrix(G).todense())
  k = np.sum(A, axis=0)
  #モジュラリティ行列B
  B = np.array([[A[i, j] - ((k[i] * k[j]) / (2 * M))  for j in range(N)] for i in range(N)])
  #モジュラリティ行列 B の固有値,固有ベクトルを求める
  values, vectors = np.linalg.eig(B)
  values = np.array([i.real for i in values])
  vectors = np.array([[vectors[i][j].real for j in range(N)]for i in range(N)])
  #vectors = np.array([[0 if abs(vectors[i][j]) < 1e-5 else vectors[i][j]  for j in range(N)]for i in range(N)])
  values = np.array([0 if abs(i) < 1e-5 else i for i in values])
  vectors = vectors.T
  z = list(zip(values, vectors))
  for i in values:
    if i < 0:
      Nn = Nn + 1
    if i > 0:
      Np = Np + 1
  #固有値の昇順に並べ替える
  z = sorted(z, key=lambda x :x[0])  
  values = [z[i][0] for i in range(len(z))]
  vectors = np.array([list(z[i][1]) for i in range(N)])
  Vp = np.array([list(z[i][1]) if z[i][0] > 0 else list(np.zeros(N)) for i in range(N-Np, N)]).T
  Vn = np.array([list(z[i][1]) if z[i][0] < 0 else list(np.zeros(N)) for i in range(Nn)]).T
  V = vectors.T
  Lam = np.array([[values[i] if i == j else 0 for j in  range(N)] for i in range(N)])
  Lamp = np.array([[Lam[i][j]**2 if (i == j) & (Lam[i][j] > 0) else 0 for j in  range(N-Np, N)] for i in range(N-Np, N)])
  Lamn = np.array([[Lam[i][j]**2 if (i == j) & (Lam[i][j] < 0) else 0 for j in  range(Nn)] for i in range(Nn)])
  
  Bp = Vp @ Lamp @ Vp.T
  Bn = Vn @ Lamn @ Vn.T
  
  lam_1 = 2.700000000000001   #5.1
  lam_2 = 2.700000000000001   #4.9
  #lam_1 = 5.1
  #lam_2 = 4.9
  xk = np.zeros((Np, 1))
  yk = np.zeros((Nn, 1))
  #変数定義
  s = cvxpy.Variable((N, 1))
  x = cvxpy.Variable((Np, 1))
  y = cvxpy.Variable((Nn, 1))
  #問題定義
  for i in range(10):
    obj = 0
    obj += xk.T @ Lamp @ (x - xk) + xk.T @ Lamp @ xk
    #obj += cvxpy.quad_form(y, Lamn)  #yT*Λn*y
    obj -= lam_1 * cvxpy.norm(s-1, p=1)
    obj -= lam_2 * cvxpy.norm(s+1, p=1)
    obj = cvxpy.Maximize(obj)
    constraints = [x == Vp.T @ s, -1 <= s , s<= 1]
    prob = cvxpy.Problem(obj, constraints)
    result = prob.solve()
    xk = x.value
    #結果表示
    s_value = s.value.reshape(N)
    
  group_1 = [i for i in range(G.number_of_nodes()) if s_value[i]>0]
  group_2 = [i for i in range(G.number_of_nodes()) if s_value[i]<0]
  s_label = np.zeros(N)
  for i in range(N):
    for j in group_1:
      if j == i:
        s_label[i] =  1
    for j in group_2:
      if j == i:
        s_label[i] =  -1

  """
  #グラフの可視化
  partition = [group_1, group_2]
  color_map = ['black'] * nx.number_of_nodes(G)
  counter = 0
  for c in partition :
    for n in c :
      color_map[n] = colors[counter]
    counter = counter + 1
  nx.draw_networkx_edges(G, pos)
  nx.draw_networkx_nodes(G, pos, node_color=color_map)
  nx.draw_networkx_labels(G, pos)
  plt.axis('off')
  plt.rcParams['figure.figsize'] = (10 ,10)
  plt.show()
  """

  community = list({frozenset(G.nodes[v]["community"]) for v in G})
  best_label = np.zeros(N)
  for i in range(2):
    for j in list(community[i]):
      if i == 0:
        best_label[j] = -1
      if i == 1:
        best_label[j] = 1

  nmi = normalized_mutual_info_score(best_label, s_label)
  mean += nmi
  list2.append(nmi)
  print(nmi)

mean = mean / len(seed_correct)
print(mean)

0.8347636587075117
0.8347636587075117
0.8782063702756283
0.8347636587075117
0.7577078109175857
0.9291252238549075
0.6917588737329794
0.7265356414540421
0.6349910030198422
0.7648442977525686
0.6349910030198425
0.5978208097977278
0.9291252238549075
0.6770006327837457
0.6425925571552905
0.8079616479391557
0.8782063702756283
0.6917588737329794
0.8079616479391557
0.7149460394223505
0.7648442977525686
0.6109637147284563
1.0
0.8079616479391557
0.7648442977525686
0.8347636587075117
0.7961285220549096
0.8782063702756283
0.8079616479391557
0.8079616479391557
0.7648442977525686
0.6917588737329793
0.9291252238549075
0.6917588737329794
0.8782063702756283
0.8079616479391555
1.0
0.7265356414540421
0.7149460394223504
0.7610260716417302
0.8347636587075117
0.7149460394223506
0.7149460394223505
0.8782063702756283
0.5697389043881019
0.9291252238549075
0.8585594574581797
0.7610260716417302
0.6349910030198422
0.9291252238549075
0.9291252238549075
0.6597573034496935
0.8079616479391557
0.698650799163015
0.761

In [ ]:
l =0.1
seed_correct = [6, 7, 10, 29, 39, 75, 81, 104, 136, 166, 171, 235, 248, 253, 266, 332, 337, 367, 394, 399, 400, 409, 429, 430, 449, 477, 479, 486, 487, 494, 500, 501, 521, 535, 540, 541, 552, 579, 582, 594, 600, 602, 629, 653, 657, 673, 680, 688, 689, 703, 709, 730, 738, 762, 773, 775, 852, 867, 875, 878, 880, 889, 904, 908, 918, 933, 945, 950, 955, 970, 971, 996, 1048, 1051, 1054, 1067, 1089, 1091, 1109, 1120, 1137, 1167, 1189, 1207, 1245, 1334, 1349, 1353, 1358, 1368, 1371, 1376, 1410, 1413, 1420, 1467, 1469, 1476, 1479, 1480]

while l < 100:
  mean = 0

  for seeds in seed_correct:
    G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.1, average_degree=4, min_community=50, seed=seeds)
    pos = nx.spring_layout(G)

    N = G.number_of_nodes()
    M = G.number_of_edges()
    V = np.zeros((N, N))
    Vp = np.zeros((N, N))
    Vn = np.zeros((N, N))
    Lam = np.zeros((N, N))
    Lamp = np.zeros((N, N))
    Lamn = np.zeros((N, N))
    Np = 0
    Nn = 0
    A = np.array(nx.adjacency_matrix(G).todense())
    k = np.sum(A, axis=0)
    #モジュラリティ行列B
    B = np.array([[A[i, j] - ((k[i] * k[j]) / (2 * M))  for j in range(N)] for i in range(N)])
    #モジュラリティ行列 B の固有値,固有ベクトルを求める
    values, vectors = np.linalg.eig(B)
    values = np.array([i.real for i in values])
    vectors = np.array([[vectors[i][j].real for j in range(N)]for i in range(N)])
    #vectors = np.array([[0 if abs(vectors[i][j]) < 1e-5 else vectors[i][j]  for j in range(N)]for i in range(N)])
    #values = np.array([0 if abs(i) < 1e-5 else i for i in values])
    vectors = vectors.T
    z = list(zip(values, vectors))
    for i in values:
      if i < 0:
        Nn = Nn + 1
      if i > 0:
        Np = Np + 1
    #固有値の昇順に並べ替える
    z = sorted(z, key=lambda x :x[0])  
    values = [z[i][0] for i in range(len(z))]
    vectors = np.array([list(z[i][1]) for i in range(N)])
    Vp = np.array([list(z[i][1]) if z[i][0] > 0 else list(np.zeros(N)) for i in range(N-Np, N)]).T
    Vn = np.array([list(z[i][1]) if z[i][0] < 0 else list(np.zeros(N)) for i in range(Nn)]).T
    V = vectors.T
    Lam = np.array([[values[i] if i == j else 0 for j in  range(N)] for i in range(N)])
    Lamp = np.array([[Lam[i][j]**2 if (i == j) & (Lam[i][j] > 0) else 0 for j in  range(N-Np, N)] for i in range(N-Np, N)])
    Lamn = np.array([[Lam[i][j]**2 if (i == j) & (Lam[i][j] < 0) else 0 for j in  range(Nn)] for i in range(Nn)])
  
    Bp = Vp @ Lamp @ Vp.T
    Bn = Vn @ Lamn @ Vn.T
  
    lam_1 = l
    lam_2 = l
    xk = np.zeros((Np, 1))
    yk = np.zeros((Nn, 1))
    #変数定義
    s = cvxpy.Variable((N, 1))
    x = cvxpy.Variable((Np, 1))
    y = cvxpy.Variable((Nn, 1))
    #問題定義
    for i in range(10):
      obj = 0
      obj += xk.T @ Lamp @ (x - xk) + xk.T @ Lamp @ xk
      #obj -= yk.T @ Lamn @ (y - yk) + yk.T @ Lamn @ yk
      #obj -= cvxpy.quad_form(y, Lamn)  #yT*Λn*y
      obj -= lam_1 * cvxpy.norm(s-1, p=1)
      obj -= lam_2 * cvxpy.norm(s+1, p=1)
      obj = cvxpy.Maximize(obj)
      constraints = [x == Vp.T @ s, -1 <= s , s<= 1]

      prob = cvxpy.Problem(obj, constraints)
      result = prob.solve()
      xk = x.value
      #結果表示
      s_value = s.value.reshape(N)
    
    #print(s_value)
    group_1 = [i for i in range(G.number_of_nodes()) if s_value[i]>0]
    group_2 = [i for i in range(G.number_of_nodes()) if s_value[i]<0]
    s_label = np.zeros(N)
    for i in range(N):
      for j in group_1:
        if j == i:
          s_label[i] =  1
      for j in group_2:
        if j == i:
          s_label[i] =  -1

    community = list({frozenset(G.nodes[v]["community"]) for v in G})
    best_label = np.zeros(N)
    for i in range(2):
      for j in list(community[i]):
        if i == 0:
          best_label[j] = -1
        if i == 1:
          best_label[j] = 1

    nmi = normalized_mutual_info_score(best_label, s_label)
    mean += nmi

  mean = mean / 100
  print(l, mean)
  l += 0.1

0.1 0.7786491631374237
0.2 0.7733254714198048
0.30000000000000004 0.7721572990903012
0.4 0.7754568943957857
0.5 0.7791035642290942
0.6 0.7702479342478022
0.7 0.7696803467498102
0.7999999999999999 0.7679051785910582
0.8999999999999999 0.7731320012738415
0.9999999999999999 0.7733562998153878
1.0999999999999999 0.772892655274431
1.2 0.769769902470654
1.3 0.7739275788608129
1.4000000000000001 0.7596821790578384
1.5000000000000002 0.774509180204085
1.6000000000000003 0.7703285542562947
1.7000000000000004 0.7623323579078445
1.8000000000000005 0.7744385446015747
1.9000000000000006 0.7727919075705116
2.0000000000000004 0.7764978027501293
2.1000000000000005 0.7689837486551234
2.2000000000000006 0.7775625504166142
2.3000000000000007 0.7722546764487426
2.400000000000001 0.7724072385353145
2.500000000000001 0.78134366511702
2.600000000000001 0.7755873703909812
2.700000000000001 0.7770159577507161
2.800000000000001 0.7770232876753336
2.9000000000000012 0.7733302742614607
3.0000000000000013 0.773912

In [ ]:
mean = 0
for seeds in seed_correct:
  G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.9, average_degree=4, min_community=50, seed=seeds)

  N = G.number_of_nodes()
  M = G.number_of_edges()
  lst_m = greedy_modularity_communities(G, resolution=0.7)

  #モジュラリティ最適化の関数を用いるときに使う
  s = np.zeros(N)
  counter=0
  for c in lst_m :
    for n in c :
      if counter==0:
        s[n] = 1
      if counter==1:
        s[n] = -1
    counter = counter + 1
  #print('s:', s)
  
  community = list({frozenset(G.nodes[v]["community"]) for v in G})
  best_label = np.zeros(N)
  for i in range(2):
    for j in list(community[i]):
      if i == 0:
        best_label[j] = -1
      if i == 1:
        best_label[j] = 1
  #print(best_label)

  nmi = normalized_mutual_info_score(best_label, s)
  mean += nmi

mean = mean / len(seed_correct)
print(mean)

In [ ]:
mean = 0

seed_correct = [6, 7, 10, 29, 39, 75, 81, 104, 136, 166, 171, 235, 248, 253, 266, 332, 337, 367, 394, 399, 400, 409, 429, 430, 449, 477, 479, 486, 487, 494, 500, 501, 521, 535, 540, 541, 552, 579, 582, 594, 600, 602, 629, 653, 657, 673, 680, 688, 689, 703, 709, 730, 738, 762, 773, 775, 852, 867, 875, 878, 880, 889, 904, 908, 918, 933, 945, 950, 955, 970, 971, 996, 1048, 1051, 1054, 1067, 1089, 1091, 1109, 1120, 1137, 1167, 1189, 1207, 1245, 1334, 1349, 1353, 1358, 1368, 1371, 1376, 1410, 1413, 1420, 1467, 1469, 1476, 1479, 1480]
for seeds in seed_correct:
  G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.1, average_degree=4, min_community=50, seed=seeds)

  N = G.number_of_nodes()
  M = G.number_of_edges()
  A = np.array(nx.adjacency_matrix(G).todense())
  k = np.sum(A, axis=0)
  #モジュラリティ行列B
  B = np.array([[A[i, j] - ((k[i] * k[j]) / (2 * M))  for j in range(N)] for i in range(N)])
  #モジュラリティ行列 B の固有値,固有ベクトルを求める
  values, vectors = np.linalg.eig(B)
  vectors = vectors.T
  z = list(zip(values, vectors))
  #固有値の昇順に並べ替える
  z = sorted(z, key=lambda x :x[0])  
  v_max = z[-1][1]
  s = np.zeros(N)
  for i in range(len(v_max)):
    if v_max[i] < 0:
      s[i] = -1
    if v_max[i] > 0:
      s[i] = 1
  print(s)
  group1 = []
  group2 = []
  for i in range(len(s)):
    if s[i] == -1:
      group1.append(i)
    if s[i] == 1:
      group2.append(i)
  lst_m = [group1, group2]
  print('ネットワーク分割結果：')
  print(lst_m)

  community = list({frozenset(G.nodes[v]["community"]) for v in G})
  best_label = np.zeros(N)
  for i in range(2):
    for j in list(community[i]):
      if i == 0:
        best_label[j] = -1
      if i == 1:
        best_label[j] = 1

  nmi = normalized_mutual_info_score(best_label, s)
  mean += nmi

mean = mean / len(seed_correct)
print(mean)

In [ ]:
"""
means = 1
seed_correct = [6, 7, 10, 29, 39, 75, 81, 104, 136, 166, 171, 235, 248, 253, 266, 332, 337, 367, 394, 399, 400, 409, 429, 430, 449, 477, 479, 486, 487, 494, 500, 501, 521, 535, 540, 541, 552, 579, 582, 594, 600, 602, 629, 653, 657, 673, 680, 688, 689, 703, 709, 730, 738, 762, 773, 775, 852, 867, 875, 878, 880, 889, 904, 908, 918, 933, 945, 950, 955, 970, 971, 996, 1048, 1051, 1054, 1067, 1089, 1091, 1109, 1120, 1137, 1167, 1189, 1207, 1245, 1334, 1349, 1353, 1358, 1368, 1371, 1376, 1410, 1413, 1420, 1467, 1469, 1476, 1479, 1480]
while means >= 0.75:
  mean = 0
  for seeds in seed_correct:
    G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.1, average_degree=4, min_community=50, seed=seeds)
    N = G.number_of_nodes()
    M = G.number_of_edges()
    lst_b = kernighan_lin_bisection(G)
    s = np.zeros(N)
    counter=0
    for c in lst_b :
      for n in c :
        if counter==0:
          s[n] = 1
        if counter==1:
          s[n] = -1
      counter = counter + 1
    #print('s:', s)
    community = list({frozenset(G.nodes[v]["community"]) for v in G})
    best_label = np.zeros(N)
    for i in range(2):
      for j in list(community[i]):
        if i == 0:
          best_label[j] = -1
        if i == 1:
          best_label[j] = 1

    nmi = normalized_mutual_info_score(best_label, s)
    mean += nmi
  means = mean / len(seed_correct)
  print(means)
"""
mean = 0
seed_correct = [7, 10, 29, 39, 51, 75, 81, 96, 104, 136, 156, 171, 235, 248, 253, 266, 294, 325, 332, 337, 345, 367, 394, 399, 400, 409, 429, 430, 449, 477, 479, 486, 487, 494, 500, 501, 506, 509, 535, 540, 541, 552, 579, 582, 583, 602, 619, 629, 653, 657, 665, 673, 680, 688, 689, 709, 728, 730, 738, 762, 775, 850, 852, 875, 878, 880, 889, 891, 904, 908, 933, 945, 949, 950, 955, 960, 971, 996, 1021, 1048, 1054, 1067, 1109, 1115, 1120, 1137, 1167, 1189, 1200, 1207, 1217, 1284, 1349, 1353, 1358, 1368, 1371, 1376, 1388, 1410]
for seeds in seed_correct:
  G = nx.LFR_benchmark_graph(100, 2, 1.5, 0.9, average_degree=4, min_community=50, seed=seeds)
  N = G.number_of_nodes()
  M = G.number_of_edges()
  lst_b = kernighan_lin_bisection(G)
  s = np.zeros(N)
  counter=0
  for c in lst_b :
    for n in c :
      if counter==0:
        s[n] = 1
      if counter==1:
        s[n] = -1
    counter = counter + 1
  #print('s:', s)
  community = list({frozenset(G.nodes[v]["community"]) for v in G})
  best_label = np.zeros(N)
  for i in range(2):
    for j in list(community[i]):
      if i == 0:
        best_label[j] = -1
      if i == 1:
        best_label[j] = 1

  nmi = normalized_mutual_info_score(best_label, s)
  mean += nmi
means = mean / len(seed_correct)
print(means)